In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import wikipedia

# Constants

In [2]:
MODIFIER_SCORE_MAPPINGS = {
    'DNS': 0,
    'NC': 0,
    'Ret': 0,
    'DNQ': -5,
    'DNPQ': -5,
    'DSQ': -10,
    'C': 0,
    'DNP': 0,
    'EX': 0,
    'DNA': 0,
    'WD': 0,
    'P': 10,
    'F': 5,
    'PF': 15,
}

POSITION_POINT_MAPPING = {
    '1': 25,
    '2': 18,
    '3': 15,
    '4': 12,
    '5': 10,
    '6': 8,
    '7': 6,
    '8': 4,
    '9': 2,
    '10': 1
}

In [11]:
def parse_table(raw_table, scores_or_teams="scores"):
    """
    Parses HTML table into a reasonable/interpretable
    pandas dataframe.
    """
    table_rows = raw_table.find_all('tr')

    l = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text for tr in td]
        l.append(row)
    if scores_or_teams == "scores":
        # header's first col is treated as index, so we skip that
        header = [th.text.rstrip() for th in table_rows[0].find_all('th')][1:]
    else:
        header = ['Entrant', 'Chassis', 'Power unit',
                  'No.', 'Driver name', 'Rounds']

    try:
        parsed_df = pd.DataFrame(l, columns=header)
    except:
        if scores_or_teams != 'scores':
            header = ['Entrant', 'Constructor', 'Chassis', 'Power unit',
                      'No.', 'Driver name', 'Rounds']
            parsed_df = pd.DataFrame(l, columns = header)
        else:
            header = header[:-1]
            parsed_df = pd.DataFrame(l, columns = header)
        
    return parsed_df


def map_outcome_to_score(race_outcome: str) -> int:
    '''
    Crunches Fantasy points from race outcome.
    INPUT
        race_outcome: representation of race outcome (e.g. "4P")
    OUTPUT
        race_score: score for racer for race
    '''
    position = ""
    modifiers = ""
    for char in race_outcome:
        if char.isdigit():
            position += char
        else:
            modifiers += char

    # This just ignores modifiers it doesn't have handling for
    modifier_score = MODIFIER_SCORE_MAPPINGS.get(modifiers, 0)
    position_score = POSITION_POINT_MAPPING.get(str(position), 0)
    race_score = modifier_score + position_score

    return race_score


def map_outcome_to_position(race_outcome: str) -> int:
    '''
    Crunches Fantasy points from race outcome.
    INPUT
        race_outcome: representation of race outcome (e.g. "4P")
    OUTPUT
        race_position: score for racer for race
    '''
    position = ""
    for char in race_outcome:
        if char.isdigit():
            position += char
    if position:
        return int(position)
    else:
        return 10000


def score_dataframe_cleanup(dirty_df):
    '''
    Cleans up the text in the dataframe
    and removes e.g. empty rows.
    '''
    score_df = dirty_df.copy()
    try:
        score_df = score_df.dropna().drop('Points', axis=1)
    except:
        score_df = score_df.dropna()
        
    position_df = score_df.copy()
    
    for col in score_df.columns:
        score_df[col] = score_df[col].apply(lambda x: x.replace("\n", ""))
        position_df[col] = position_df[col].apply(lambda x: x.replace("\n", ""))
        if col != 'Driver':
            score_df[col] = score_df[col].map(map_outcome_to_score)
            position_df[col] = position_df[col].map(map_outcome_to_position)

    return score_df, position_df


def team_dataframe_cleanup(dirty_df):
    df = dirty_df.dropna(axis=0)
    df = df.loc[:, ["Driver name", "Entrant"]]
    for col in df.columns:
        df[col] = df[col].apply(lambda x: x.replace("\n", ""))

    return df


def get_data_by_year(year):
    """
    Pulls in the raw HTML table of F1 results from Wikipedia.
    """
    wiki = wikipedia.WikipediaPage(f"{year} Formula One World Championship")
    soup = BeautifulSoup(wiki.html())
    tables = soup.find_all('table', {'class': 'wikitable'})

    raw_score_table = tables[-4]
    raw_team_table = tables[0]

    score_table = parse_table(raw_score_table, "scores")
    score_df, position_df = score_dataframe_cleanup(score_table)

    
    team_table = parse_table(raw_team_table, "teams")
    team_df = team_dataframe_cleanup(team_table)

    position_teammates_df = get_teammate_mappings(position_df, team_df)
    score_teammates_final_df = update_scores_by_comparison(position_teammates_df, score_df)
    
    score_teammates_final_df['Year'] = year
    
    return score_teammates_final_df


def split_drivers(team_members, drivers):
    results = []
    for d in drivers:
        if d in team_members:
            results.append(d)
            team_members = team_members.replace(d, "")
    else:
        return results

def get_teammate(driver, members):
    for m in members:
        if driver in m:
            teammate = [person for person in m if person != driver][0]
            return teammate
    
def get_teammate_mappings(score_df, team_df):
    teammates = []
    # Break string of members into separate drivers
    drivers = score_df.Driver.values
    team_df2 = team_df.copy()
    team_df2["Driver name"] = team_df2["Driver name"].map(
        lambda x: split_drivers(x, drivers))

    score_teammate_df = score_df.copy()
    score_teammate_df["teammate"] = None
    for idx, row in score_teammate_df.iterrows():
        score_teammate_df.loc[idx, "teammate"] = get_teammate(
            row.Driver, 
            team_df2["Driver name"].values
        )
    return score_teammate_df


def update_scores_by_comparison(position_teammates_df, score_df):
    final_score_df = score_df.copy()
    
    for row_index, (_ ,row) in enumerate(position_teammates_df.iterrows()):
        teammate = row.teammate
        teammate_row = position_teammates_df.loc[position_teammates_df.Driver==teammate]
        
        for col_index, column in enumerate(position_teammates_df.columns):
            if column not in ['Driver', 'teammate']:
                racer_position = row[column] 
                teammate_position = teammate_row[column].iloc[0]
                if racer_position < teammate_position:
                    try:
                        final_score_df.iloc[row_index, col_index] += 3
                    except:
                        print(row_index, col_index)
        
    return final_score_df

In [12]:
score_df_17 = get_data_by_year(2017)
score_df_18 = get_data_by_year(2018)
score_df_19 = get_data_by_year(2019)
score_df_20 = get_data_by_year(2020)

In [13]:
score_df_17.head(5)

,Driver,AUS,CHN,BHR,RUS,ESP,MON,CAN,AZE,AUT,...,BEL,ITA,SIN,MAL,JPN,USA,MEX,BRA,ABU,Year
1,Lewis Hamilton,21,28,21,12,28,6,28,10,12,...,28,28,28,21,28,28,2,12,18,2017
2,Sebastian Vettel,28,21,28,21,21,28,15,15,21,...,21,18,0,15,0,21,12,28,18,2017
3,Valtteri Bottas,15,8,15,28,0,15,18,21,28,...,10,18,15,10,12,10,21,21,28,2017
4,Kimi Räikkönen,12,10,12,15,0,18,6,0,10,...,12,10,0,0,13,15,18,15,12,2017
5,Daniel Ricciardo,0,12,13,0,18,18,18,28,18,...,18,15,21,15,15,0,0,8,0,2017


In [14]:
score_df_18.head(5)

,Driver,AUS,BHR,CHN,AZE,ESP,MON,CAN,FRA,AUT,...,BEL,ITA,SIN,RUS,JPN,USA,MEX,BRA,ABU,Year
1,Lewis Hamilton,21,15,12,28,28,18,10,28,0,...,21,28,28,28,28,18,15,28,28,2018
2,Sebastian Vettel,28,28,4,12,15,21,28,10,15,...,28,12,18,18,8,12,21,8,21,2018
3,Kimi Räikkönen,15,0,18,21,0,12,8,18,21,...,0,21,10,12,13,28,15,18,0,2018
4,Max Verstappen,8,0,10,0,18,2,18,21,28,...,18,13,21,13,18,21,28,21,18,2018
5,Valtteri Bottas,4,21,21,0,18,10,21,6,0,...,12,15,12,18,18,10,10,10,10,2018


In [15]:
score_df_19.head(5)

,Driver,AUS,BHR,CHN,AZE,ESP,MON,CAN,FRA,AUT,...,BEL,ITA,SIN,RUS,JPN,MEX,USA,BRA,ABU,Year
1,Lewis Hamilton,28,28,28,18,33,38,28,38,10,...,21,20,15,33,20,28,18,9,43,2019
2,Valtteri Bottas,33,18,28,38,28,15,17,18,18,...,15,21,10,18,28,15,38,0,12,2019
3,Max Verstappen,18,15,15,15,18,15,13,15,33,...,0,7,18,15,0,11,18,38,21,2019
4,Charles Leclerc,10,33,10,15,10,0,15,18,31,...,38,38,28,28,8,27,20,0,18,2019
5,Sebastian Vettel,15,10,18,18,15,21,31,15,12,...,17,0,28,0,31,21,0,3,10,2019


In [16]:
score_df_20.head(5)

,Driver,AUT,STY,HUN,GBR,70A,ESP,BEL,ITA,TUS,RUS,EIF,POR,EMI,TUR,BHR,SKH,ABU,Year
1,Lewis Hamilton,12,38,43,38,26,38,38,21,43,25,28,43,33,28,38,0,15,2020
2,Valtteri Bottas,38,18,15,0,25,20,18,13,18,33,0,18,28,0,4,17,21,2020
3,Max Verstappen,0,18,21,26,28,21,18,0,0,21,26,18,0,11,26,0,38,2020
4,Sergio Pérez,11,11,6,0,0,10,1,1,13,15,15,9,11,21,3,28,0,2020
5,Daniel Ricciardo,0,7,7,15,0,3,20,11,15,13,18,2,18,4,9,10,14,2020


In [17]:
training_data = score_df_17.merge(score_df_18, how='outer')
training_data = training_data.merge(score_df_19, how='outer')

In [20]:
training_data.tail(10)

,Driver,AUS,CHN,BHR,RUS,ESP,MON,CAN,AZE,AUT,...,SIN,MAL,JPN,USA,MEX,BRA,ABU,Year,FRA,GER
55,Lando Norris,3,0,11,4,0,0,0,4,11,...,9,NaN,0,9,0,4,7,2019,2.0,0.0
56,Kimi Räikkönen,7,5,9,3,3,3,0,4,5,...,0,NaN,3,3,0,15,3,2019,9.0,3.0
57,Daniil Kvyat,4,0,0,3,2,6,1,0,0,...,0,NaN,1,3,0,1,5,2019,0.0,18.0
58,Nico Hülkenberg,9,0,3,4,0,0,6,3,0,...,5,NaN,-10,2,1,0,0,2019,7.0,0.0
59,Lance Stroll,5,0,0,0,0,0,5,2,0,...,3,NaN,2,0,0,0,0,2019,0.0,15.0
60,Kevin Magnussen,11,0,3,5,9,0,0,3,0,...,5,NaN,0,0,3,3,3,2019,3.0,4.0
61,Antonio Giovinazzi,0,0,0,0,0,0,3,0,1,...,4,NaN,0,0,3,10,0,2019,0.0,0.0
62,Romain Grosjean,0,3,0,0,1,4,3,0,3,...,3,NaN,3,3,0,0,0,2019,0.0,9.0
63,Robert Kubica,0,0,0,0,0,0,0,0,0,...,3,NaN,0,0,0,0,0,2019,3.0,4.0
64,George Russell,3,3,3,0,3,3,3,3,3,...,0,NaN,3,3,3,3,3,2019,0.0,0.0


In [21]:
test_data = score_df_20.copy()

In [23]:
training_data.to_csv('training_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)